Wig-Wag Visualizer Python Code

This is a code that will help visualization of timing parameters.


This code will allow the user to adjust parameters for the wig wag headlights including:

Flash (hz): the rate that it will cycle left off, right on -> right on left off

Off Delay (s): the # of seconds the lights will continue flash after the horn or bell is no longer pressed


How to use:

1: Click the double play arrows in the tool bar

2: Wait for the first image to appear

3: Click the Horn/Bell button once to simulate the operator pressing and holding the bell or horn button

4: Click the Horn/Bell button again to simulate lifting the bell or horn button to start the Off Delay

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display, HTML
import io
import base64
import ipywidgets as widgets
from ipywidgets import FloatSlider, Button, Output, Label
import time
import threading

# Global variables to manage state
horn_active = False
hz = 1.0  # Initial frequency
wig_wag_delay = 5  # Initial Delay for Turning Off
stopFlag = False

# Output widget to capture button actions
out = Output()

# Label to display the status of the horn/bell
status_label = Label(value="Horn/Bell: Not Active")

def create_animation(hz, horn_active):
    global flashing_end_time
    global stopFlag
    
    # Calculate on and off times based on frequency
    on_time = 1 / (2 * hz)   # Time (in seconds) for headlights to be on
    off_time = 1 / (2 * hz)  # Time (in seconds) for headlights to be off

    # Create figure and axis
    fig, ax = plt.subplots()
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 2)
    ax.axis('off')  # Hide axes
    fig.patch.set_facecolor('black')
    ax.set_facecolor('black')

    # Create two white circles for headlights
    headlight1 = plt.Circle((2, 1), 0.5, color='white', fill=True)
    headlight2 = plt.Circle((8, 1), 0.5, color='white', fill=True)
    ax.add_patch(headlight1)
    ax.add_patch(headlight2)

    total_time = 1 / hz  # Total duration of the animation in seconds
    fps = 30            # Frames per second for the animation

    def update(frame):
        cycle_time = (frame / fps) % (2 * (on_time + off_time))
        currentTime = time.time()
        if horn_active and stopFlag == False:
            # Start the animation
            if cycle_time < on_time:
                headlight1.set_visible(True)
                headlight2.set_visible(False)
            elif cycle_time < on_time + off_time:
                headlight1.set_visible(False)
                headlight2.set_visible(True)
            elif cycle_time < 2 * on_time + off_time:
                headlight1.set_visible(True)
                headlight2.set_visible(False)
            else:
                headlight1.set_visible(False)
                headlight2.set_visible(True)
        else:
            # Continuous on when horn is not active
            headlight1.set_visible(True)
            headlight2.set_visible(True)
        
        return headlight1, headlight2

    # Prepare to save animation frames
    frames = []
    for frame in range(int(total_time * fps)):
        update(frame)
        buf = io.BytesIO()
        plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0)
        buf.seek(0)
        img = Image.open(buf)
        frames.append(img.copy())  # Ensure a copy of the image is added to the list
        buf.close()

    # Save all frames as a GIF
    gif_buffer = io.BytesIO()
    frames[0].save(
        gif_buffer,
        format='GIF',
        append_images=frames[1:],
        save_all=True,
        duration=1000 / fps,  # Duration per frame in milliseconds
        loop=0
    )
    gif_buffer.seek(0)

    # Encode GIF as base64
    gif_base64 = base64.b64encode(gif_buffer.read()).decode('utf-8')

    # Display GIF inline
    with out:
        out.clear_output()
        display(HTML(f'<img src="data:image/gif;base64,{gif_base64}" />'))

    plt.close(fig)  # Close the figure to avoid displaying it twice

def on_button_click(b):
    global horn_active, flashing_end_time, stopFlag, startingTime
    stopFlag = False
    horn_active = not horn_active  # Toggle the state

    # Update the status label
    status_label.value = "Horn/Bell: Active" if horn_active else "Horn/Bell: Not Active"

    if not horn_active:
        # Set the time when the extended flashing should end
        time.sleep(wig_wag_delay)
        stopFlag = True
    
    create_animation(hz_slider.value, horn_active)  # Update animation with current frequency and state

# Create interactive widgets
hz_slider = FloatSlider(value=1.0, min=0.1, max=2.0, step=0.01, description='Flash (Hz):')
wig_wag_delay_slider = FloatSlider(value=5, min=0, max=20, step=0.5, description='Off Delay (s):')
horn_button = Button(description="Horn/Bell")


# Register button action
horn_button.on_click(on_button_click)

# Display widgets
display(hz_slider, wig_wag_delay_slider, horn_button, status_label, out)

def update_frequency(change):
    create_animation(change['new'], horn_active)

def update_wig_wag_delay(change):
    global wig_wag_delay
    wig_wag_delay = change['new']

# Link slider to animation function
hz_slider.observe(update_frequency, names='value')
wig_wag_delay_slider.observe(update_wig_wag_delay, names='value')

# Create initial animation
create_animation(hz_slider.value, horn_active)


FloatSlider(value=1.0, description='Flash (Hz):', max=2.0, min=0.1, step=0.01)

FloatSlider(value=5.0, description='Off Delay (s):', max=20.0, step=0.5)

Button(description='Horn/Bell', style=ButtonStyle())

Label(value='Horn/Bell: Not Active')

Output()